# Humor Classifier
- Inspired by the following resources:
    - https://huggingface.co/docs/transformers/training
    - https://huggingface.co/docs/transformers/tasks/sequence_classification
    - https://medium.com/mlearning-ai/fine-tuning-bert-for-tweets-classification-ft-hugging-face-8afebadd5dbf

In [ ]:
!pip install datasets --quiet
!pip install transformers --quiet
!pip install evaluate --quiet
!pip install torch --quiet

In [1]:
from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import get_scheduler

import evaluate

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

from tqdm.auto import tqdm

### Load Data

In [2]:
dataset = load_dataset('csv', data_files={'train': './data/oasst1_train_cleaned.csv', 'test': './data/oasst1_test_cleaned.csv'})

Found cached dataset csv (C:/Users/timon/.cache/huggingface/datasets/csv/default-5f0235340c70a9e3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

### Prepare Data

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [4]:
def transform_labels(data):
    humor = data['humor']
    label = 1 if humor > 0 else 0
    return {'labels': label}

dataset = dataset.map(transform_labels)

Loading cached processed dataset at C:\Users\timon\.cache\huggingface\datasets\csv\default-5f0235340c70a9e3\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-78fb1b57e2d32411.arrow
Loading cached processed dataset at C:\Users\timon\.cache\huggingface\datasets\csv\default-5f0235340c70a9e3\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-a068dc0ffb2ded18.arrow


In [5]:
def tokenize_data(data):
    return tokenizer(data['text'], padding="max_length", truncation=True)

remove_columns = ['message_id', 'role', 'lang', 'humor', 'text']
dataset = dataset.map(tokenize_data, remove_columns=remove_columns, batched=True)

Map:   0%|          | 0/82933 [00:00<?, ? examples/s]

Map:   0%|          | 0/4333 [00:00<?, ? examples/s]

In [6]:
dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

### Training- and Validationsplit

In [7]:
train_validation_dataset = dataset['train']
test_dataset = dataset['test']

In [8]:
train_validation_dataset = train_validation_dataset.train_test_split(test_size=0.1)
train_dataset = train_validation_dataset['train']
validation_dataset = train_validation_dataset['test']

In [9]:
small_train_dataset = train_dataset.shuffle(seed=42).select(range(1000))
small_eval_dataset = validation_dataset.shuffle(seed=42).select(range(1000))

### Data Loader

In [10]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

### Setup Model

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [12]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

### Training

In [13]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [14]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/375 [00:00<?, ?it/s]

### Evaluate

In [ ]:
metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()